In [7]:
import pandas as pd

routes = pd.read_csv('../OpenTransitDelhiBuses/routes.txt')
stop_times = pd.read_csv('../OpenTransitDelhiBuses/stop_times.txt')
stops = pd.read_csv('../OpenTransitDelhiBuses/stops.txt')
trips = pd.read_csv('../OpenTransitDelhiBuses/trips.txt')

In [8]:
print('routes')
display(routes.head())

print('stop_times')
display(stop_times.head())

print('stops')
display(stops.head())

print('trips')
display(trips.head())

routes


,agency_id,route_id,route_long_name,route_short_name,route_type
0,DIMTS,142,828AUP,NaN,3
1,DIMTS,10001,971DOWN,NaN,3
2,DIMTS,362,961DOWN,NaN,3
3,DIMTS,274,819DOWN,NaN,3
4,DIMTS,10386,824STLDOWN2,NaN,3


stop_times


,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,142_12_50,12:50:00,12:50:00,146,0
1,142_12_50,12:51:11,12:51:11,148,1
2,142_12_50,12:52:49,12:52:49,149,2
3,142_12_50,12:53:54,12:53:54,488,3
4,142_12_50,12:55:44,12:55:44,233,4


stops


,stop_code,stop_id,stop_lat,stop_lon,stop_name,zone_id
0,DC4539,0,28.851958,77.088107,Narela Terminal,0
1,DC4540,1,28.853311,77.088548,Police Station Narela,1
2,DC4541,2,28.854965,77.089322,Safiyabad Crossing,2
3,DC4542,3,28.857713,77.092846,Ramdev Chowk / Pitori Johad,3
4,DC4543,4,28.857684,77.097117,Narela A-6 / CPJ College,4


trips


,route_id,service_id,trip_id,shape_id
0,142,1,142_12_50,NaN
1,142,1,142_11_17,NaN
2,142,1,142_09_14,NaN
3,142,1,142_14_00,NaN
4,142,1,142_10_44,NaN


In [9]:
# trips = trips.sort_values(by=['trip_id'])
trips_prep = trips.drop(['shape_id'],axis=1)
stops_prep = stops.copy()
stop_times_prep = stop_times.copy()
routes_prep = routes.drop(['route_short_name'],axis=1)

In [10]:
stop_times_prep[stop_times_prep['trip_id']=='142_12_50']

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,142_12_50,12:50:00,12:50:00,146,0
1,142_12_50,12:51:11,12:51:11,148,1
2,142_12_50,12:52:49,12:52:49,149,2
3,142_12_50,12:53:54,12:53:54,488,3
4,142_12_50,12:55:44,12:55:44,233,4
5,142_12_50,12:57:10,12:57:10,915,5
6,142_12_50,12:58:43,12:58:43,916,6
7,142_12_50,13:00:46,13:00:46,2161,7
8,142_12_50,13:04:51,13:04:51,2162,8
9,142_12_50,13:06:08,13:06:08,3569,9


In [11]:
def fix_invalid_time(time_str):
    """Fix time strings where hours exceed 23 by rolling over to the next day."""
    try:
        h, m, s = map(int, time_str.split(':'))
        if h >= 24:
            h = h % 24  # Convert 24+ hours to valid format
        return f"{h:02}:{m:02}:{s:02}"
    except ValueError:
        return None

# calculate the dwell time
stop_times_prep['arrival_time'] = stop_times_prep['arrival_time'].astype(str).apply(fix_invalid_time)
stop_times_prep['departure_time'] = stop_times_prep['departure_time'].astype(str).apply(fix_invalid_time)

stop_times_prep['arrival_time'] = pd.to_datetime(stop_times_prep['arrival_time'], format='%H:%M:%S')
stop_times_prep['departure_time'] = pd.to_datetime(stop_times_prep['departure_time'], format='%H:%M:%S')
stop_times_prep['dwell_time'] = (stop_times_prep['departure_time'] - stop_times_prep['arrival_time']).dt.total_seconds()

# get the hour
stop_times_prep['hour'] = stop_times_prep['arrival_time'].dt.hour


In [ ]:
# get distance between stops
stop_times_prep['distance_between_stops'] = stop_times_prep.groupby('trip_id')['shape_dist_traveled'].diff().fillna(0)
stop_times_prep 

In [13]:
stop_times_prep.stop_id.unique()

array([  146,   148,   149, ..., 13407, 11745, 22704], dtype=int64)

In [14]:
# remove date from arrival time and departure time
stop_times_prep['arrival_time'] = stop_times_prep['arrival_time'].dt.time
stop_times_prep['departure_time'] = stop_times_prep['departure_time'].dt.time

# remove unwanted columns
# stop_times_prep = stop_times_prep.drop(['pickup_type','drop_off_type','timepoint'],axis=1)

# add route id and service id to dataset
stop_times_prep = stop_times_prep.merge(trips[['route_id', 'service_id', 'trip_id']], on='trip_id', how='left')

stop_times_prep

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,dwell_time,hour,route_id,service_id
0,142_12_50,12:50:00,12:50:00,146,0,0.0,12,142,1
1,142_12_50,12:51:11,12:51:11,148,1,0.0,12,142,1
2,142_12_50,12:52:49,12:52:49,149,2,0.0,12,142,1
3,142_12_50,12:53:54,12:53:54,488,3,0.0,12,142,1
4,142_12_50,12:55:44,12:55:44,233,4,0.0,12,142,1
...,...,...,...,...,...,...,...,...,...
3724315,674_05_16,06:43:50,06:43:50,166,35,0.0,6,674,1
3724316,674_05_16,06:44:42,06:44:42,167,36,0.0,6,674,1
3724317,674_05_16,06:46:37,06:46:37,168,37,0.0,6,674,1
3724318,674_05_16,06:48:14,06:48:14,169,38,0.0,6,674,1


In [15]:
stops

,stop_code,stop_id,stop_lat,stop_lon,stop_name,zone_id
0,DC4539,0,28.851958,77.088107,Narela Terminal,0
1,DC4540,1,28.853311,77.088548,Police Station Narela,1
2,DC4541,2,28.854965,77.089322,Safiyabad Crossing,2
3,DC4542,3,28.857713,77.092846,Ramdev Chowk / Pitori Johad,3
4,DC4543,4,28.857684,77.097117,Narela A-6 / CPJ College,4
...,...,...,...,...,...,...
10554,247F0F67,22753,28.583989,77.327651,Noida Sector 20 Poilce Station,22753
10555,7187C89B,22754,28.582006,77.324835,Staff Quarter,22754
10556,4D05E2EB,22755,28.602100,77.322206,Vasundhra Enclave Sports Complex,22755
10557,69DFCF3D,22756,28.600819,77.326879,Mayur Vihar Phase-3 Xing BRD,22756


In [16]:
interconnected_routes = stop_times_prep.groupby('stop_id')['route_id'].nunique()
interconnected_stops = interconnected_routes[interconnected_routes > 1].index
interconnected_routes_data = stop_times_prep[stop_times_prep['stop_id'].isin(interconnected_stops)]

interconnected_routes_data


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,dwell_time,hour,route_id,service_id
0,142_12_50,12:50:00,12:50:00,146,0,0.0,12,142,1
1,142_12_50,12:51:11,12:51:11,148,1,0.0,12,142,1
2,142_12_50,12:52:49,12:52:49,149,2,0.0,12,142,1
3,142_12_50,12:53:54,12:53:54,488,3,0.0,12,142,1
4,142_12_50,12:55:44,12:55:44,233,4,0.0,12,142,1
...,...,...,...,...,...,...,...,...,...
3724315,674_05_16,06:43:50,06:43:50,166,35,0.0,6,674,1
3724316,674_05_16,06:44:42,06:44:42,167,36,0.0,6,674,1
3724317,674_05_16,06:46:37,06:46:37,168,37,0.0,6,674,1
3724318,674_05_16,06:48:14,06:48:14,169,38,0.0,6,674,1


In [17]:
bus_data =  interconnected_routes_data.copy()
bus_data = bus_data.merge(stops[['stop_id', 'stop_lat', 'stop_lon']], on='stop_id', how='left')
bus_data

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,dwell_time,hour,route_id,service_id,stop_lat,stop_lon
0,142_12_50,12:50:00,12:50:00,146,0,0.0,12,142,1,28.614614,76.978024
1,142_12_50,12:51:11,12:51:11,148,1,0.0,12,142,1,28.614481,76.981062
2,142_12_50,12:52:49,12:52:49,149,2,0.0,12,142,1,28.614150,76.985217
3,142_12_50,12:53:54,12:53:54,488,3,0.0,12,142,1,28.611746,76.985721
4,142_12_50,12:55:44,12:55:44,233,4,0.0,12,142,1,28.610319,76.981338
...,...,...,...,...,...,...,...,...,...,...,...
3669612,674_05_16,06:43:50,06:43:50,166,35,0.0,6,674,1,28.620876,77.049338
3669613,674_05_16,06:44:42,06:44:42,167,36,0.0,6,674,1,28.621124,77.051543
3669614,674_05_16,06:46:37,06:46:37,168,37,0.0,6,674,1,28.621946,77.056321
3669615,674_05_16,06:48:14,06:48:14,169,38,0.0,6,674,1,28.623017,77.060310


In [42]:
bus_data.route_id.value_counts()

route_id
5722     20352
5721     18762
5254     16320
5431     13720
5895     13694
         ...  
1523        12
10520       11
1667         6
10612        5
10576        4
Name: count, Length: 2403, dtype: int64

In [ ]:
bus_data

array([  142, 10001,   362, ...,  6001, 10669, 10670], dtype=int64)

In [23]:
# filter out 3 routes
bus_data_1 = bus_data.copy()
bus_data_1 = bus_data_1[bus_data_1['route_id'].isin([5721 , 5722 , 5254])]

bus_data_1

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,dwell_time,hour,route_id,service_id,stop_lat,stop_lon
2203828,5254_11_40,11:40:00,11:40:00,12722,0,0.0,11,5254,1,28.674286,77.287117
2203829,5254_11_40,11:41:47,11:41:47,10100,1,0.0,11,5254,1,28.673043,77.282749
2203830,5254_11_40,11:43:10,11:43:10,10101,2,0.0,11,5254,1,28.670400,77.280900
2203831,5254_11_40,11:44:25,11:44:25,10102,3,0.0,11,5254,1,28.667850,77.282250
2203832,5254_11_40,11:44:25,11:44:25,10102,4,0.0,11,5254,1,28.667850,77.282250
...,...,...,...,...,...,...,...,...,...,...,...
3131571,5722_19_48,21:33:37,21:33:37,14425,60,0.0,21,5722,1,28.659391,77.318398
3131572,5722_19_48,21:39:05,21:39:05,10060,61,0.0,21,5722,1,28.647061,77.317903
3131573,5722_19_48,21:40:48,21:40:48,11475,62,0.0,21,5722,1,28.643371,77.316656
3131574,5722_19_48,21:41:31,21:41:31,11214,63,0.0,21,5722,1,28.641950,77.315817


In [24]:
# calculate the travel time 
bus_data_2 = bus_data_1.copy()
bus_data_2 = bus_data_2.sort_values(by=['trip_id', 'stop_sequence'])

bus_data_2['arrival_time'] = pd.to_datetime(bus_data_2['arrival_time'], format='%H:%M:%S')
bus_data_2['travel_time'] = bus_data_2.groupby('trip_id')['arrival_time'].diff().dt.total_seconds()
bus_data_2['arrival_time'] = bus_data_2['arrival_time'].dt.time
# bus_data_2['travel_time'] = bus_data_2.groupby('trip_id')['travel_time'].apply(lambda x: x.fillna(0))
bus_data_2['travel_time'].fillna(0, inplace=True)
bus_data_2


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10876\2203721219.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  bus_data_2['travel_time'].fillna(0, inplace=True)


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,dwell_time,hour,route_id,service_id,stop_lat,stop_lon,travel_time
2211688,5254_04_50,04:50:00,04:50:00,12722,0,0.0,4,5254,1,28.674286,77.287117,0.0
2211689,5254_04_50,04:51:47,04:51:47,10100,1,0.0,4,5254,1,28.673043,77.282749,107.0
2211690,5254_04_50,04:53:10,04:53:10,10101,2,0.0,4,5254,1,28.670400,77.280900,83.0
2211691,5254_04_50,04:54:25,04:54:25,10102,3,0.0,4,5254,1,28.667850,77.282250,75.0
2211692,5254_04_50,04:54:25,04:54:25,10102,4,0.0,4,5254,1,28.667850,77.282250,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3129843,5722_22_32,00:17:37,00:17:37,14425,60,0.0,0,5722,1,28.659391,77.318398,56.0
3129844,5722_22_32,00:23:05,00:23:05,10060,61,0.0,0,5722,1,28.647061,77.317903,328.0
3129845,5722_22_32,00:24:48,00:24:48,11475,62,0.0,0,5722,1,28.643371,77.316656,103.0
3129846,5722_22_32,00:25:31,00:25:31,11214,63,0.0,0,5722,1,28.641950,77.315817,43.0


In [25]:
from geopy.distance import geodesic

bus_data_3 = bus_data_2.copy()
def compute_distance(group):
    distances = [0]  
    for i in range(1, len(group)):
        coords_prev = (group.iloc[i-1]['stop_lat'], group.iloc[i-1]['stop_lon'])
        coords_curr = (group.iloc[i]['stop_lat'], group.iloc[i]['stop_lon'])
        distance = geodesic(coords_prev, coords_curr).meters
        distances.append(distance)
    group['distance_between_stops'] = distances
    return group

bus_data_3 = bus_data_3.groupby('trip_id', group_keys=False).apply(compute_distance)
bus_data_3


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10876\2622641925.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bus_data_3 = bus_data_3.groupby('trip_id', group_keys=False).apply(compute_distance)


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,dwell_time,hour,route_id,service_id,stop_lat,stop_lon,travel_time,distance_between_stops
2211688,5254_04_50,04:50:00,04:50:00,12722,0,0.0,4,5254,1,28.674286,77.287117,0.0,0.000000
2211689,5254_04_50,04:51:47,04:51:47,10100,1,0.0,4,5254,1,28.673043,77.282749,107.0,448.627344
2211690,5254_04_50,04:53:10,04:53:10,10101,2,0.0,4,5254,1,28.670400,77.280900,83.0,344.221128
2211691,5254_04_50,04:54:25,04:54:25,10102,3,0.0,4,5254,1,28.667850,77.282250,75.0,311.906714
2211692,5254_04_50,04:54:25,04:54:25,10102,4,0.0,4,5254,1,28.667850,77.282250,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3129843,5722_22_32,00:17:37,00:17:37,14425,60,0.0,0,5722,1,28.659391,77.318398,56.0,235.407911
3129844,5722_22_32,00:23:05,00:23:05,10060,61,0.0,0,5722,1,28.647061,77.317903,328.0,1367.391281
3129845,5722_22_32,00:24:48,00:24:48,11475,62,0.0,0,5722,1,28.643371,77.316656,103.0,426.748766
3129846,5722_22_32,00:25:31,00:25:31,11214,63,0.0,0,5722,1,28.641950,77.315817,43.0,177.572319


In [ ]:
# save the dataset
# bus_data_3.to_csv('../OpenTransitDelhiBuses/delhi_static_bus_data.csv',index=False)

In [26]:
# crete edge dataset
bus_data_4 = bus_data_3.copy()
bus_data_4.sort_values(by=['trip_id', 'stop_sequence'], inplace=True)

# Create edges by shifting stop_id
bus_data_4['next_stop_id'] = bus_data_4.groupby('trip_id')['stop_id'].shift(-1)
bus_data_4['edge_distance'] = bus_data_4.groupby('trip_id')['distance_between_stops'].shift(-1)
bus_data_4['edge_travel_time'] = bus_data_4.groupby('trip_id')['travel_time'].shift(-1)

# Drop rows where next_stop_id is NaN (last stop of each trip)
bus_edges = bus_data_4.dropna(subset=['next_stop_id'])

# Convert next_stop_id to int
bus_edges['next_stop_id'] = bus_edges['next_stop_id'].astype(int)

# Keep relevant columns
bus_edges = bus_edges[['trip_id', 'stop_id', 'next_stop_id', 'edge_distance', 'edge_travel_time']]
bus_edges


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10876\889591600.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bus_edges['next_stop_id'] = bus_edges['next_stop_id'].astype(int)


,trip_id,stop_id,next_stop_id,edge_distance,edge_travel_time
2211688,5254_04_50,12722,10100,448.627344,107.0
2211689,5254_04_50,10100,10101,344.221128,83.0
2211690,5254_04_50,10101,10102,311.906714,75.0
2211691,5254_04_50,10102,10102,0.000000,0.0
2211692,5254_04_50,10102,10104,600.587982,144.0
...,...,...,...,...,...
3129842,5722_22_32,10059,14425,235.407911,56.0
3129843,5722_22_32,14425,10060,1367.391281,328.0
3129844,5722_22_32,10060,11475,426.748766,103.0
3129845,5722_22_32,11475,11214,177.572319,43.0


In [27]:
bus_edges.isna().sum()

trip_id             0
stop_id             0
next_stop_id        0
edge_distance       0
edge_travel_time    0
dtype: int64

In [ ]:
# # save
# bus_edges.to_csv('../OpenTransitDelhiBuses/edge_data.csv',index=False)

In [30]:
# read dataset
import pandas as pd
bus_data_3 = pd.read_csv('../OpenTransitDelhiBuses/delhi_static_bus_data.csv')
bus_data_3

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,dwell_time,hour,route_id,service_id,stop_lat,stop_lon,travel_time,distance_between_stops
0,5254_04_50,04:50:00,04:50:00,12722,0,0.0,4,5254,1,28.674286,77.287117,0.0,0.000000
1,5254_04_50,04:51:47,04:51:47,10100,1,0.0,4,5254,1,28.673043,77.282749,107.0,448.627344
2,5254_04_50,04:53:10,04:53:10,10101,2,0.0,4,5254,1,28.670400,77.280900,83.0,344.221128
3,5254_04_50,04:54:25,04:54:25,10102,3,0.0,4,5254,1,28.667850,77.282250,75.0,311.906714
4,5254_04_50,04:54:25,04:54:25,10102,4,0.0,4,5254,1,28.667850,77.282250,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55429,5722_22_32,00:17:37,00:17:37,14425,60,0.0,0,5722,1,28.659391,77.318398,56.0,235.407911
55430,5722_22_32,00:23:05,00:23:05,10060,61,0.0,0,5722,1,28.647061,77.317903,328.0,1367.391281
55431,5722_22_32,00:24:48,00:24:48,11475,62,0.0,0,5722,1,28.643371,77.316656,103.0,426.748766
55432,5722_22_32,00:25:31,00:25:31,11214,63,0.0,0,5722,1,28.641950,77.315817,43.0,177.572319


In [45]:
bus_data_3.trip_id.unique()

array(['5254_04_50', '5254_05_00', '5254_05_10', '5254_05_19',
       '5254_05_20', '5254_05_25', '5254_05_30', '5254_05_35',
       '5254_05_40', '5254_05_45', '5254_05_50', '5254_06_00',
       '5254_06_01', '5254_06_10', '5254_06_20', '5254_06_25',
       '5254_06_30', '5254_06_40', '5254_06_50', '5254_06_53',
       '5254_06_56', '5254_07_00', '5254_07_10', '5254_07_11',
       '5254_07_12', '5254_07_13', '5254_07_20', '5254_07_25',
       '5254_07_28', '5254_07_29', '5254_07_35', '5254_07_37',
       '5254_07_40', '5254_07_44', '5254_07_45', '5254_07_47',
       '5254_07_50', '5254_07_52', '5254_08_00', '5254_08_05',
       '5254_08_10', '5254_08_13', '5254_08_16', '5254_08_20',
       '5254_08_23', '5254_08_25', '5254_08_28', '5254_08_32',
       '5254_08_36', '5254_08_40', '5254_08_41', '5254_08_44',
       '5254_08_45', '5254_08_48', '5254_08_50', '5254_08_52',
       '5254_08_55', '5254_08_59', '5254_09_00', '5254_09_04',
       '5254_09_08', '5254_09_10', '5254_09_12', '5254_

In [32]:
route_5254 = bus_data_3[bus_data_3['route_id']==5254]
route_5721 = bus_data_3[bus_data_3['route_id']==5721]
route_5722 = bus_data_3[bus_data_3['route_id']==5722]

In [ ]:
route_5254['trip_id'] = route_5254['trip_id'].str.split('_', n=1).str[1]
route_5721['trip_id'] = route_5721['trip_id'].str.split('_', n=1).str[1]
route_5722['trip_id'] = route_5722['trip_id'].str.split('_', n=1).str[1]

In [51]:
common_values = set(route_5722['trip_id']) & set(route_5721['trip_id']) & set(route_5254['trip_id'])
len(common_values)

138

In [53]:
# filter datasets
route_5722 = route_5722[route_5722['trip_id'].isin(common_values)]
route_5721 = route_5721[route_5721['trip_id'].isin(common_values)]
route_5254 = route_5254[route_5254['trip_id'].isin(common_values)]

In [54]:
# encode trip ids
all_trip_ids = pd.concat([route_5722['trip_id'], route_5721['trip_id'], route_5254['trip_id']]).unique()
trip_id_map = {trip_id: idx for idx, trip_id in enumerate(all_trip_ids)}

route_5722['encoded_trip_id'] = route_5722['trip_id'].map(trip_id_map)
route_5721['encoded_trip_id'] = route_5721['trip_id'].map(trip_id_map)
route_5254['encoded_trip_id'] = route_5254['trip_id'].map(trip_id_map)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10876\1808504741.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_5722['encoded_trip_id'] = route_5722['trip_id'].map(trip_id_map)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10876\1808504741.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_5254['encoded_trip_id'] = route_5254['trip_id'].map(trip_id_map)


In [ ]:
# # save dataset
# route_5254.to_csv('../OpenTransitDelhiBuses/route_5254.csv',index=False)
# route_5721.to_csv('../OpenTransitDelhiBuses/route_5721.csv',index=False)
# route_5722.to_csv('../OpenTransitDelhiBuses/route_5722.csv',index=False)

In [58]:
bus_edges['route_id'] = bus_edges['trip_id'].str.split('_').str[0].astype(int)

bus_edges_5254 = bus_edges[bus_edges['route_id'] == 5254]
bus_edges_5722 = bus_edges[bus_edges['route_id'] == 5722]
bus_edges_5721 = bus_edges[bus_edges['route_id'] == 5721]

In [ ]:
# encode the trip id
bus_edges_5722 = bus_edges_5722[bus_edges_5722['trip_id'].isin(common_values)]
bus_edges_5721 = bus_edges_5721[bus_edges_5721['trip_id'].isin(common_values)]
bus_edges_5254 = bus_edges_5254[bus_edges_5254['trip_id'].isin(common_values)]

bus_edges_5254['trip_id'] = bus_edges_5254['trip_id'].str.split('_', n=1).str[1]
bus_edges_5721['trip_id'] = bus_edges_5721['trip_id'].str.split('_', n=1).str[1]
bus_edges_5722['trip_id'] = bus_edges_5722['trip_id'].str.split('_', n=1).str[1]

bus_edges_5254['encoded_trip_id'] = bus_edges_5254['trip_id'].map(trip_id_map)
bus_edges_5722['encoded_trip_id'] = bus_edges_5722['trip_id'].map(trip_id_map)
bus_edges_5721['encoded_trip_id'] = bus_edges_5721['trip_id'].map(trip_id_map)

In [68]:
bus_edges_5254.isna().sum()

trip_id             0
stop_id             0
next_stop_id        0
edge_distance       0
edge_travel_time    0
route_id            0
encoded_trip_id     0
dtype: int64

In [ ]:
# # save dataset
# bus_edges_5254.to_csv('../OpenTransitDelhiBuses/bus_edges_5254.csv',index=False)
# bus_edges_5721.to_csv('../OpenTransitDelhiBuses/bus_edges_5721.csv',index=False)
# bus_edges_5722.to_csv('../OpenTransitDelhiBuses/bus_edges_5722.csv',index=False)

In [70]:
route_5254

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,dwell_time,hour,route_id,service_id,stop_lat,stop_lon,travel_time,distance_between_stops,encoded_trip_id
0,04_50,04:50:00,04:50:00,12722,0,0.0,4,5254,1,28.674286,77.287117,0.0,0.000000,0
1,04_50,04:51:47,04:51:47,10100,1,0.0,4,5254,1,28.673043,77.282749,107.0,448.627344,0
2,04_50,04:53:10,04:53:10,10101,2,0.0,4,5254,1,28.670400,77.280900,83.0,344.221128,0
3,04_50,04:54:25,04:54:25,10102,3,0.0,4,5254,1,28.667850,77.282250,75.0,311.906714,0
4,04_50,04:54:25,04:54:25,10102,4,0.0,4,5254,1,28.667850,77.282250,0.0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15775,20_44,22:25:55,22:25:55,12725,55,0.0,22,5254,1,28.560350,77.164417,110.0,460.631233,137
15776,20_44,22:27:13,22:27:13,12726,56,0.0,22,5254,1,28.560417,77.161100,78.0,324.647832,137
15777,20_44,22:28:43,22:28:43,12727,57,0.0,22,5254,1,28.559967,77.157317,90.0,373.505537,137
15778,20_44,22:29:21,22:29:21,12728,58,0.0,22,5254,1,28.558700,77.158117,38.0,160.764971,137
